# Training

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM
from sklearn.model_selection import train_test_split

import pickle

pickle_in = open('/home/trojan/Desktop/defect-detection/conv1d/training data/X.pickle', 'rb')
X = pickle.load(pickle_in)

pickle_in = open('/home/trojan/Desktop/defect-detection/conv1d/training data/y.pickle', 'rb')
y = pickle.load(pickle_in)

# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, shuffle= True)

print (X.shape[1:])
model = Sequential()

model.add(LSTM(1, activation='relu', input_shape=(499, 1), return_sequences = True))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=X.shape[1:]))
#model.add(LSTM(1, activation='relu', return_sequences = False))

#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

model.add(Dense(100, activation='relu'))
          
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=5) #validation_split=0.15)

(499, 1)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/5
17850/17850 [==============================] - 94s 5ms/sample - loss: 0.3181 - acc: 0.8735
Epoch 2/5
17850/17850 [==============================] - 93s 5ms/sample - loss: 0.1131 - acc: 0.9612
Epoch 3/5
17850/17850 [==============================] - 93s 5ms/sample - loss: 0.0566 - acc: 0.9796
Epoch 4/5
17850/17850 [==============================] - 92s 5ms/sample - loss: 0.0311 - acc: 0.9893
Epoch 5/5
17850/17850 [==============================] - 94s 5ms/sample - loss: 0.0175 - acc: 0.9946


# Evaluation on split data

In [3]:
# evaluate model
model.evaluate(x_valid, y_valid, batch_size=1, verbose=1)


3150/3150 [==============================] - 85s 27ms/sample - loss: 0.0304 - acc: 0.9886


[0.03042299077826112, 0.9885714]

In [4]:
pred = model.predict(x_valid, verbose = 1)

3150/3150 [==============================] - 3s 868us/sample


In [5]:
from keras.models import load_model 
model.save("lstm_model.h5")
model.save_weights("lstm_weights.h5")

Using TensorFlow backend.


In [6]:
import numpy as np

labels =  ['defected', 'nodefect']
 
indices = np.round(model.predict(x_valid),1)

In [7]:
import pandas as pd
# Get classes by np.round
# Get filenames (set shuffle=false in generator is important)
#filenames=test_generator.filenames

y = [labels[int(i[0])] for i in indices]
results=pd.DataFrame({"prediction": pred [:,0],
                          "class_index": indices[:,0],
                          "class_name": y })

results.to_csv("/home/trojan/Desktop/defect-detection/LSTM/results_valid.csv",index=False)

# Evaluation on new test data

In [8]:
#prediction is fine

import pandas as pd
import numpy as np
import os
import glob

path = '/home/trojan/Desktop/defect-detection/conv1d/test data 2'
os.chdir (path)

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

for test_csv in all_filenames:
    test_csv =pd.read_csv(test_csv)
    series = np.asarray(test_csv)
    series = np.array(series).reshape(-1, 499, 1)
#test_image = test_image.astype('float32')

    pred=model.predict(series, verbose=1)
    indices = np.round(pred[:,0])
    print (indices)
#labels =  ['defected', 'nodefect']

#indices2 = np.argmax(model.predict(test_csv),1)
                     
#print([labels[x] for x in indices2])

1/1 [==============================] - 0s 886us/sample
[0.]
1/1 [==============================] - 0s 854us/sample
[0.]
1/1 [==============================] - 0s 829us/sample
[0.]
1/1 [==============================] - 0s 738us/sample
[0.]
1/1 [==============================] - 0s 1ms/sample
[0.]


In [10]:
#on 222 test samples

pickle_in = open('/home/trojan/Desktop/defect-detection/conv1d/test data/X_test2.pickle', 'rb')
X_test = pickle.load(pickle_in)

pickle_in = open('/home/trojan/Desktop/defect-detection/conv1d/test data/y_test2.pickle', 'rb')
y_test = pickle.load(pickle_in)

model.evaluate(X_test, y_test, batch_size=1, verbose=1)


222/222 [==============================] - 5s 22ms/sample - loss: 0.0426 - acc: 0.9910


[0.04257032950786746, 0.990991]

In [84]:
pickle_in = open('/home/trojan/Desktop/defect-detection/conv1d/test data/X_test.pickle', 'rb')
X_test = pickle.load(pickle_in)

pred = model.predict(X_test, verbose = 1)

222/222 [==============================] - 0s 204us/sample


In [85]:
import numpy as np
import pandas as pd

labels =  ['defected', 'nodefect']
 
indices = np.round(model.predict(X_test),0)
#preds_cls_idx = pred.argmax(axis=-1)

for i in indices:
    y = (labels[int(i[0])])
    print (y)

    '''results=pd.DataFrame({"prediction": pred [:,0],
                          "class_index": indices[:,0],
                          "class_name":  y })

results.to_csv("/home/trojan/Desktop/defect-detection/conv1d/results222_test.csv",index=False)'''

nodefect
defected
nodefect
nodefect
nodefect
defected
defected
defected
defected
nodefect
defected
defected
nodefect
defected
defected
nodefect
defected
nodefect
defected
defected
nodefect
nodefect
nodefect
defected
defected
nodefect
defected
defected
defected
defected
defected
defected
defected
nodefect
defected
defected
defected
nodefect
nodefect
nodefect
nodefect
defected
defected
nodefect
nodefect
defected
nodefect
nodefect
defected
defected
nodefect
nodefect
nodefect
nodefect
nodefect
nodefect
nodefect
defected
nodefect
defected
nodefect
nodefect
defected
nodefect
defected
defected
nodefect
defected
defected
defected
defected
defected
nodefect
defected
defected
nodefect
nodefect
defected
defected
defected
defected
nodefect
nodefect
nodefect
defected
defected
defected
nodefect
defected
nodefect
nodefect
defected
defected
nodefect
nodefect
nodefect
nodefect
defected
nodefect
defected
defected
defected
nodefect
nodefect
nodefect
nodefect
nodefect
defected
nodefect
defected
defected
n

In [86]:
import pandas as pd
# Get classes by np.round
# Get filenames (set shuffle=false in generator is important)
#filenames=test_generator.filenames
    
#for f in all_filenames:

y = [labels[int(i[0])] for i in indices]
#y = [labels[x] for x in preds_cls_idx]
results=pd.DataFrame({"prediction": pred [:,0],
                          "class_index": indices[:,0],
                          "class_name":  y })

results.to_csv("/home/trojan/Desktop/defect-detection/conv1d/results222_test.csv",index=False)

In [9]:
import pandas as pd
import numpy as np
import os
import glob
import csv

path = '/home/trojan/Desktop/defect-detection/conv1d/test data2'
os.chdir (path)

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

results = pd.DataFrame()
for test_csv in all_filenames:
    df =pd.read_csv(test_csv)
    series = np.asarray(df)
    series = np.array(series).reshape(-1, 499, 1)
#test_image = test_image.astype('float32')

    pred=model.predict(series, verbose=1)
    indices = np.round(pred)
    print (indices)
    labels =  ['defected', 'nodefect']

    ''''for i in indices:
        y = (labels[int(i)])
        print (y)'''
    y = [labels[int(i)] for i in indices]
    print (y)
    print (test_csv)
    #M.append(test_csv)

#print (M)
    df=pd.DataFrame({"file": test_csv,
                          "prediction": pred [:,0],
                          "class_index": indices [:,0],
                          "class_name":  y })
    
    results = results.append(df)
    
        
results.to_csv("/home/trojan/Desktop/defect-detection/LSTM/results222_test.csv",index=False)
    

1/1 [==============================] - 0s 24ms/sample
[[1.]]
['nodefect']
point(1511,1261).csv
1/1 [==============================] - 0s 22ms/sample
[[1.]]
['nodefect']
point(1510,1271).csv
1/1 [==============================] - 0s 23ms/sample
[[0.]]
['defected']
point(1550,1042).csv
1/1 [==============================] - 0s 23ms/sample
[[0.]]
['defected']
point(1551,1033).csv
1/1 [==============================] - 0s 23ms/sample
[[1.]]
['nodefect']
point(1511,1286).csv
1/1 [==============================] - 0s 23ms/sample
[[0.]]
['defected']
point(1550,1031).csv
1/1 [==============================] - 0s 22ms/sample
[[1.]]
['nodefect']
point(1510,1266).csv
1/1 [==============================] - 0s 22ms/sample
[[1.]]
['nodefect']
point(1510,1277).csv
1/1 [==============================] - 0s 23ms/sample
[[0.]]
['defected']
point(1551,1012).csv
1/1 [==============================] - 0s 23ms/sample
[[0.]]
['defected']
point(1550,1026).csv
1/1 [==============================] - 0s 27ms/sam

[[1.]]
['nodefect']
point(1510,1275).csv
1/1 [==============================] - 0s 22ms/sample
[[0.]]
['defected']
point(1551,1017).csv
1/1 [==============================] - 0s 22ms/sample
[[0.]]
['defected']
point(1551,1013).csv
1/1 [==============================] - 0s 22ms/sample
[[1.]]
['nodefect']
point(1510,1297).csv
1/1 [==============================] - 0s 23ms/sample
[[1.]]
['nodefect']
point(1509,1295).csv
1/1 [==============================] - 0s 23ms/sample
[[1.]]
['nodefect']
point(1511,1275).csv
1/1 [==============================] - 0s 22ms/sample
[[1.]]
['nodefect']
point(1511,1264).csv
1/1 [==============================] - 0s 23ms/sample
[[1.]]
['nodefect']
point(1510,1262).csv
1/1 [==============================] - 0s 22ms/sample
[[1.]]
['nodefect']
point(1511,1291).csv
1/1 [==============================] - 0s 22ms/sample
[[0.]]
['defected']
point(1551,1007).csv
1/1 [==============================] - 0s 26ms/sample
[[1.]]
['nodefect']
point(1511,1294).csv
1/1 [====

1/1 [==============================] - 0s 22ms/sample
[[0.]]
['defected']
point(1550,1037).csv
1/1 [==============================] - 0s 22ms/sample
[[1.]]
['nodefect']
point(1511,1252).csv
1/1 [==============================] - 0s 21ms/sample
[[1.]]
['nodefect']
point(1510,1260).csv
1/1 [==============================] - 0s 23ms/sample
[[1.]]
['nodefect']
point(1511,1268).csv
1/1 [==============================] - 0s 22ms/sample
[[0.]]
['defected']
point(1551,1040).csv
1/1 [==============================] - 0s 23ms/sample
[[0.]]
['defected']
point(1550,1047).csv
1/1 [==============================] - 0s 22ms/sample
[[1.]]
['nodefect']
point(1510,1280).csv
1/1 [==============================] - 0s 22ms/sample
[[0.]]
['defected']
point(1550,1017).csv
1/1 [==============================] - 0s 23ms/sample
[[1.]]
['nodefect']
point(1550,1045).csv
1/1 [==============================] - 0s 21ms/sample
[[1.]]
['nodefect']
point(1511,1269).csv
1/1 [==============================] - 0s 22ms/sam

# with timeseriesgen

In [35]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.python.keras.utils.data_utils import Sequence
from keras.utils import Sequence

DATADIR = "/home/trojan/Desktop/defect-detection/conv1d/test data2"
#CATEGORIES = ["defected", "nodefect"]




#for category in CATEGORIES:
    #path = os.path.join(DATADIR, category)
    #for csv in os.listdir(path):
os.chdir (DATADIR)

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
for f in all_filenames:
    df = pd.read_csv(f)
    #print (df)
    series = np.array(df)
    targets = np.array(df)
    test_generator = TimeseriesGenerator(series, targets, length = 499,
                               batch_size=1, start_index = 0, end_index = 499)

print('Samples: %d' % len(test_generator))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [46]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.python.keras.utils.data_utils import Sequence
from keras.utils import Sequence
    
test_generator = TimeseriesGenerator(X_test, X_test, length = len(test_generator),
                               batch_size=1, start_index = 0, end_index = 499)

print('Samples: %d' % len(test_generator))

Samples: 1


In [47]:

#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
#test_generator.reset()

pred=model.predict_generator(test_generator,
verbose=1)

IndexError: index 499 is out of bounds for axis 0 with size 222